<a href="https://colab.research.google.com/github/nurbekzhussip/nlp_labs/blob/master/lab_6/Lab_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import json

with open('dataset_43428_1.txt', encoding="utf-8") as json_file:
    dataset = json.load(json_file)


import nltk
nltk.download('punkt')
nltk.download('stopwords')
from itertools import combinations
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, RegexpTokenizer,word_tokenize
from nltk.stem.snowball import RussianStemmer
from tokenize import tokenize, untokenize, NUMBER, STRING, NAME, OP
from pymystem3 import Mystem
from nltk.cluster.util import cosine_distance
import networkx as nx
import numpy as np
import re, string, random



def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix

def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)


stemmer = RussianStemmer()
mystem = Mystem()
tokenizer = RegexpTokenizer(r'\w+')
stop_words_ru = stopwords.words('russian')
stop_words_ru.extend(['что', 'но','это','однако', 'так', 'вот', 'быть', 'как', 'в', '—', '–', 'к', 'на', '...'])

cleared_text=[]
cleared_sentences = []
cleared_words = []

for text in dataset:
  sentences_list = sent_tokenize(text)

  for sentences in sentences_list:  
    words = word_tokenize(sentences)
    for word in words:
      if (word not in stop_words_ru):
        cleared_words.append(mystem.lemmatize(word))
        cleared_words.append(word)
    cleared_sentences.append(' '.join(cleared_words))
    cleared_words = []
      
  cleared_text.append(' '.join(cleared_sentences))
  cleared_sentences=[]

summarize_all_text =[]
summarize_text =[]

for text in dataset:
  sentences_list = sent_tokenize(text)
  sentence_similarity_martix = build_similarity_matrix(sentences_list, stop_words_ru)

  # Step 3 - Rank sentences in similarity martix
  sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
  scores = nx.pagerank(sentence_similarity_graph)


  # Step 4 - Sort the rank and pick top sentences
  ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences_list)), reverse=True)    
  #print("Indexes of top ranked_sentence order are ", ranked_sentence)  

  top_n = round(len(sentences_list)/2)

  for i in range(top_n):
    summarize_text.append(ranked_sentence[i][1])
  
  summarize_all_text.append(summarize_text)
  summarize_text = []

with open('results.json', 'w', encoding='utf-8') as f:
    json.dump(summarize_all_text, f, ensure_ascii=False, indent=4)

  Cloning https://github.com/nlpub/pymystem3 to /tmp/pip-req-build-1a1i9_9l
  Running command git clone -q https://github.com/nlpub/pymystem3 /tmp/pip-req-build-1a1i9_9l
  Created wheel for pymystem3: filename=pymystem3-0.2.0-cp36-none-any.whl size=9921 sha256=fb41821098f63de29401f75cf99c408e26ba1b9c03051d2c6bd6f79049432ee8
  Stored in directory: /tmp/pip-ephem-wheel-cache-s0zi6fff/wheels/7d/75/c2/216a594291dee680749ce12c60d16125cfe1f363059e7163dc
Successfully built pymystem3
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
